In [1]:
import time

import imageio
import collections

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

import PIL.Image
from io import BytesIO
from IPython.display import clear_output, Image, display

%matplotlib inline

In [2]:
def animate(mass, position):
    image = np.zeros((1024, 1024))
    position = np.round(position).astype(np.int32)
    for i in range(len(mass)):
        temp_position = position[i]
        temp_mass = mass[i][0]
        
        image[np.maximum(temp_position[0] - temp_mass//2, 0) : np.minimum(temp_position[0] + temp_mass//2, 1024), 
              np.maximum(temp_position[1] - temp_mass//2, 0) : np.minimum(temp_position[1] + temp_mass//2, 1024)] = 255
        image = np.uint8(np.clip(image, 0, 255))
    
    file = BytesIO()
    PIL.Image.fromarray(image).save(file, 'jpeg')
    clear_output(wait = True)
    display(Image(data=file.getvalue()))

    return image

In [3]:
DIMENSIONS = 2

In [4]:
f_initial = tf.placeholder(shape=[None, DIMENSIONS], dtype=tf.float32)
v_initial = tf.placeholder(shape=[None, DIMENSIONS], dtype=tf.float32)
x_initial = tf.placeholder(shape=[None, DIMENSIONS], dtype=tf.float32)

f = tf.Variable(f_initial, dtype=tf.float32, validate_shape=False)

m = tf.placeholder(shape=[None, 1], dtype=tf.float32)

a = tf.Variable(tf.zeros_like(f_initial), validate_shape=False)
v = tf.Variable(v_initial, dtype=tf.float32, validate_shape=False)
x = tf.Variable(x_initial, dtype=tf.float32, validate_shape=False)

t_h = tf.placeholder(shape=[], dtype=tf.float32)
t_s = tf.placeholder(shape=[], dtype=tf.int32)

t = tf.Variable(0, dtype=tf.int32)

In [5]:
def loop_condition(i, p):
    return i < p.t_s

def loop_body(i, p):
    is_collide = tf.logical_or(p.x < p.m / 2.0, p.x > 1024 - p.m / 2.0)

    f_ = p.f * tf.pow(-1.0, tf.cast(is_collide, dtype=tf.float32))
    
    a_ = f_ / p.m

    v_ = p.v * tf.pow(-1.0, tf.cast(is_collide, dtype=tf.float32)) + a_ * p.t_h
    
    x_ = p.x + v_ * p.t_h

    return i + 1, Pair(f_, p.m, a_, v_, x_, p.t_h, p.t_s)

Pair = collections.namedtuple('Pair', 'f, m, a, v, x, t_h, t_s')
loop_vars = (t, Pair(f, m, a, v, x, t_h, t_s))

simulate = tf.while_loop(loop_condition, loop_body, loop_vars=loop_vars)

In [6]:
start_time = time.time()

force_0 = np.array([[0.1, 0.0], [0.0, 0.1], [0.1, 0.1], [0.1, 0.0], [0.0, 0.1], [0.1, 0.1]], dtype=np.float32)
velocity_0 = np.array([[0.1, 0], [5, 0.0], [2.5, 0.2], [0.1, 0], [5, 0.0], [2.5, 0.2]], dtype=np.float32)
position_0 = np.array([[512, 512], [128, 128], [256, 256], [512, 512], [128, 128], [256, 256]])

mass = [[10], [5], [2], [10], [5], [2]]
time_horizon = 0.02
number_horizon = int(100 / time_horizon)

gif_frames = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer(), 
             feed_dict={f_initial:force_0, 
                        v_initial:velocity_0, 
                        x_initial:position_0})
    print(sess.run(simulate, feed_dict={m:mass, t_h:time_horizon, t_s:number_horizon})) 

execution_time = time.time() - start_time
print(execution_time)

(5000, Pair(f=array([[0.1, 0. ],
       [0. , 0.1],
       [0.1, 0.1],
       [0.1, 0. ],
       [0. , 0.1],
       [0.1, 0.1]], dtype=float32), m=array([[10.],
       [ 5.],
       [ 2.],
       [10.],
       [ 5.],
       [ 2.]], dtype=float32), a=array([[0.01, 0.  ],
       [0.  , 0.02],
       [0.05, 0.05],
       [0.01, 0.  ],
       [0.  , 0.02],
       [0.05, 0.05]], dtype=float32), v=array([[1.0999572, 0.       ],
       [5.       , 1.9998825],
       [7.4996376, 5.1998053],
       [1.0999572, 0.       ],
       [5.       , 1.9998825],
       [7.4996376, 5.1998053]], dtype=float32), x=array([[572.00836, 512.     ],
       [627.9951 , 228.01776],
       [756.0319 , 526.0453 ],
       [572.00836, 512.     ],
       [627.9951 , 228.01776],
       [756.0319 , 526.0453 ]], dtype=float32), t_h=0.02, t_s=5000))
0.28153371810913086
